<a href="https://colab.research.google.com/github/felixM2020/NBA_Team_Comparison/blob/main/NBA_scraper_vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NBA Team Data Visualisation

##Scrape data

###Import Scraping Tools

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

###Create Soup

In [3]:
link = 'https://www.espn.com/nba/stats/team/_/season/2022/seasontype/2/table/offensive/sort/avgPoints/dir/desc'
r = requests.get(link)
soup = BeautifulSoup(r.content, 'html.parser')


###Scrape table rows

In [4]:
def find_teams(table):
  table_container = soup.find('table', {'class': 'Table Table--align-right Table--fixed Table--fixed-left'})

  teams_container = table_container.find('tbody', {'class':'Table__TBODY'})
  teams = teams_container.find_all('div', {'class':'flex items-start mr7'})
  return [x.text for x in teams]

def row_to_array(row_values):
  return [x.text for x in row_values]

def find_values(table):
  values_container = soup.find('div', {'class':'Table__Scroller'})
  values = values_container.find_all('tr', {'class': 'Table__TR Table__TR--sm Table__even'})
  return [row_to_array(x.find_all('td', {'class':'Table__TD'})) for x in values]




table = soup.find('div', {'class': 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
teams = find_teams(table)
values = find_values(table)
rows = []
for i in range(30):
  team = teams[i]
  stats = values[i]
  stats.insert(0,team)
  rows.append(stats)

for row in rows:
  print(row)




['Minnesota Timberwolves', '82', '115.9', '41.6', '91.0', '45.7', '14.8', '41.3', '35.8', '18.0', '23.1', '77.8', '11.2', '32.9', '44.2', '25.7', '8.8', '5.6', '13.7', '21.8']
['Memphis Grizzlies', '82', '115.6', '43.5', '94.4', '46.1', '11.5', '32.7', '35.3', '17.0', '23.1', '73.4', '14.1', '35.0', '49.2', '26.0', '9.8', '6.5', '12.5', '19.8']
['Milwaukee Bucks', '82', '115.5', '41.8', '89.4', '46.8', '14.1', '38.4', '36.6', '17.8', '22.9', '77.6', '10.3', '36.5', '46.7', '23.9', '7.6', '4.0', '12.7', '18.2']
['Charlotte Hornets', '82', '115.3', '42.8', '91.4', '46.8', '13.9', '38.2', '36.5', '15.8', '21.4', '74.0', '10.8', '33.7', '44.6', '28.1', '8.6', '4.9', '12.7', '19.9']
['Phoenix Suns', '82', '114.8', '43.7', '90.1', '48.5', '11.6', '31.9', '36.4', '15.9', '19.9', '79.7', '9.8', '35.5', '45.3', '27.4', '8.6', '4.4', '12.3', '19.9']
['Atlanta Hawks', '82', '113.9', '41.5', '88.3', '47.0', '12.9', '34.4', '37.4', '18.1', '22.3', '81.2', '10.0', '33.9', '44.0', '24.6', '7.2', '4.2

### Scrape headers

In [5]:

def find_headers(table):
  values_container = soup.find('div', {'class':'Table__Scroller'})
  headers = values_container.find_all('th', {'class': 'Table__TH'})
  return [x.text for x in headers]

cols = ['Team']
headers = find_headers(table)
[cols.append(x) for x in headers]
cols


['Team',
 'GP',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OR',
 'DR',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TO',
 'PF']

###Combine headers and rows into a df

In [6]:
df = pd.DataFrame(rows, columns = cols)
df = df.set_index('Team')
for i in range(0, len(df.columns)):
  col = df.columns[i]
  df[col] = df[col].apply(lambda x : float(x))
print(df.dtypes)

GP     float64
PTS    float64
FGM    float64
FGA    float64
FG%    float64
3PM    float64
3PA    float64
3P%    float64
FTM    float64
FTA    float64
FT%    float64
OR     float64
DR     float64
REB    float64
AST    float64
STL    float64
BLK    float64
TO     float64
PF     float64
dtype: object


##Prepare Data for Visualisation

In [7]:
from scipy import stats

norm = df
for col in df.columns:
  norm[col] = pd.Series(stats.zscore(norm[col]))
norm = norm.drop('GP', axis = 1)

#Visualise Data

### All Teams

In [15]:
import plotly.graph_objects as go
fig = go.Figure()
team = [0,0]
cols = norm.columns
teams = norm.index
for i in range(len(norm.index)):
  X = norm.iloc[i,:].values.flatten().tolist()
  fig.add_trace(go.Scatterpolar(
        r=X,
        theta=cols,
        fill='toself',
        name = norm.index[i]
      ))

                  
fig.update_layout(
      polar=dict(
          radialaxis=dict(
            visible=True,
            range=[-4, 4]
              )),

        showlegend=True,
        title = 'NBA League Wide Team Comparison'
      )


fig.show()

Store individual plots in dictionary

In [16]:
import plotly.graph_objects as go
team_traces = {}
teams = norm.index
cols = norm.columns
for i in range(len(norm.index)):
  X = norm.iloc[i,:].values.flatten().tolist()
  name = teams[i]
  team_traces[name] = {
      'r':X,
      'theta':cols,
      'name' : teams[i]}

def to_polar(dic_entry):
  return go.Scatterpolar(
        r=dic_entry['r'],
        theta=dic_entry['theta'],
        fill='toself',
        name = dic_entry['name']
      )


Create Figure and buttons

In [24]:
from numpy import true_divide
fig = go.Figure()
for i in range(len(norm.index)):
  X = norm.iloc[i,:].values.flatten().tolist()
  fig.add_trace(go.Scatterpolar(
        r=X,
        theta=cols,
        fill='toself',
        name = norm.index[i]
      ))

def visible(i,but):
  team[but] = i
  out = []
  for b in range(30):
    if b == team[0] or b == team[1]:
      out.append(True)
    else:
      out.append(False)
  return out

buttons = list()
for i in range(len(teams)):
  current = team_traces[teams[i]]
  x = current['r']
  visible = [False for c in range(30)]
  visible[i] = True
  button = dict(label = teams[i],
                       method="update",
                       args=[{'visible': visible, 'name': current['name'],'type' : 'scatterpolar', 'fill':'toself'},
                             [i]
                        ]  
                  )
  buttons.append(button)




Add buttons and labels

In [25]:
updatemenu = []
first_menu = dict(x = 0.1, y = 0.1,
            active=0,
            buttons= buttons, showactive = True
             )
updatemenu.append(first_menu)

fig.update_layout(
    updatemenus=updatemenu,
        annotations = [
                       dict(text="Team 2", x=0.1, xref="paper", y=0.1, yref="paper",
                             align="left", showarrow=False),
                       dict(text="Team 1", x=0.1, xref="paper", y=0.53, yref="paper",
                             align="left", showarrow=False), 
                       ]
            ,

    


      polar=dict(
          radialaxis=dict(
            visible=True,
            range=[-4, 4]
              )),

        showlegend=False,
      title = 'Team Comparisons'
      )


fig.show()